Download modelo

In [1]:
!wget -O model_last.safetensors "https://huggingface.co/Tharyck/audios-multispeaker-refine/resolve/main/model_last.safetensors"
!wget -O vocab.txt "https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/vocab.txt"

--2025-06-28 14:57:16--  https://huggingface.co/Tharyck/audios-multispeaker-refine/resolve/main/model_last.safetensors
Resolving huggingface.co (huggingface.co)... 3.163.189.114, 3.163.189.37, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.114|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /Tharyck/multispeaker-ptbr-f5tts/resolve/main/model_last.safetensors [following]
--2025-06-28 14:57:16--  https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/model_last.safetensors
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/9f/c8/9fc843f07f25545540816d5d615555591a0e2a404bb3c24c1f965f5d098acd1b/39435f0d0179abc166cd443c6bc60b8a55f7908e368b58887853e4a7d94d488b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model_last.safetensors%3B+filename%3D%22model_last.safetensors%22%3B&Expires=1751126236&Policy=eyJ

Configuraçoes para o modelo

In [2]:
!mkdir configs
!mkdir audio_ref
!touch input.txt
!echo "[F034-0960] Este é um teste de referência." > input.txt
!wget -O configs/F5TTS_v1_Base.yaml "https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/configs/F5TTS_v1_Base.yaml"
!wget -O audio_ref/F034-0960.wav https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/audio_ref/F034-0960.wav
!wget -O audio_ref/F034-0960.txt https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/audio_ref/F034-0960.txt

--2025-06-28 14:57:23--  https://huggingface.co/Tharyck/multispeaker-ptbr-f5tts/resolve/main/configs/F5TTS_v1_Base.yaml
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.74, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /api/resolve-cache/models/Tharyck/multispeaker-ptbr-f5tts/984a19879e2fcff4a1db586f6a74994c9dfed537/configs%2FF5TTS_v1_Base.yaml?%2FTharyck%2Fmultispeaker-ptbr-f5tts%2Fresolve%2Fmain%2Fconfigs%2FF5TTS_v1_Base.yaml=&etag=%22c7717facb114c0c1fc598e29a5c589485249d9d1%22 [following]
--2025-06-28 14:57:23--  https://huggingface.co/api/resolve-cache/models/Tharyck/multispeaker-ptbr-f5tts/984a19879e2fcff4a1db586f6a74994c9dfed537/configs%2FF5TTS_v1_Base.yaml?%2FTharyck%2Fmultispeaker-ptbr-f5tts%2Fresolve%2Fmain%2Fconfigs%2FF5TTS_v1_Base.yaml=&etag=%22c7717facb114c0c1fc598e29a5c589485249d9d1%22
Reusing existing connection to huggingface.co:4

In [3]:
  !apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [4]:
!pip install f5-tts==1.1.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 66.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Reason for being yanked: Buggy for custom model usage
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.5 MB/s eta 

In [18]:
import os
import subprocess
import torch

# 1. Obtém o local de instalação da lib via pip show
def get_install_path(package):
    result = subprocess.run(["pip", "show", package], capture_output=True, text=True)
    for line in result.stdout.splitlines():
        if line.startswith("Location:"):
            return line.split(":", 1)[1].strip()
    raise RuntimeError(f"Não foi possível encontrar o caminho de instalação de {package}")

install_path = get_install_path("f5-tts")
utils_path = os.path.join(install_path, "f5_tts", "model", "utils.py")

print("Caminho detectado:", utils_path)

# 2. Função a ser adicionada
code_to_add = """

# get the empirically pruned step for sampling
def get_epss_timesteps(n, device, dtype):
    dt = 1 / 32
    predefined_timesteps = {
        5: [0, 2, 4, 8, 16, 32],
        6: [0, 2, 4, 6, 8, 16, 32],
        7: [0, 2, 4, 6, 8, 16, 24, 32],
        10: [0, 2, 4, 6, 8, 12, 16, 20, 24, 28, 32],
        12: [0, 2, 4, 6, 8, 10, 12, 14, 16, 20, 24, 28, 32],
        16: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32],
    }
    t = predefined_timesteps.get(n, [])
    if not t:
        return torch.linspace(0, 1, n + 1, device=device, dtype=dtype)
    return dt * torch.tensor(t, device=device, dtype=dtype)
"""

# 3. Adiciona ao final do utils.py
with open(utils_path, "a") as f:
    f.write(code_to_add)

print("✅ Função get_epss_timesteps adicionada com sucesso!")


Caminho detectado: /usr/local/lib/python3.11/dist-packages/f5_tts/model/utils.py
✅ Função get_epss_timesteps adicionada com sucesso!


In [19]:
import os
import subprocess

# 1. Função para obter o caminho de instalação da lib
def get_install_path(package):
    result = subprocess.run(["pip", "show", package], capture_output=True, text=True)
    for line in result.stdout.splitlines():
        if line.startswith("Location:"):
            return line.split(":", 1)[1].strip()
    raise RuntimeError(f"Não foi possível encontrar o caminho de instalação de {package}")

# 2. Caminho completo para o arquivo cfm.py
install_path = get_install_path("f5-tts")
cfm_path = os.path.join(install_path, "f5_tts", "model", "cfm.py")

print("Editando:", cfm_path)

# 3. Lê o arquivo original
with open(cfm_path, "r") as f:
    lines = f.readlines()

# 4. Substitui a linha-alvo e insere o novo bloco com indentação apropriada
new_lines = []
for line in lines:
    if "t = torch.linspace(t_start, 1, steps + 1" in line:
        indent = line[:len(line) - len(line.lstrip())]  # Captura a indentação da linha
        new_lines.append(f"{indent}if t_start == 0:\n")
        new_lines.append(f"{indent}    t = get_epss_timesteps(steps, device=self.device, dtype=step_cond.dtype)\n")
    else:
        new_lines.append(line)

# 5. Reescreve o arquivo com as alterações
with open(cfm_path, "w") as f:
    f.writelines(new_lines)

print("✅ Linha substituída com sucesso!")


Editando: /usr/local/lib/python3.11/dist-packages/f5_tts/model/cfm.py
✅ Linha substituída com sucesso!


In [5]:
# Remove possíveis versões quebradas
!pip uninstall -y numpy torch torchvision torchaudio transformers accelerate

# Instala versões compatíveis
!pip install numpy==1.26.4
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers accelerate


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver 

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/90

In [1]:
import torch
import numpy as np
import transformers

print("CUDA disponível:", torch.cuda.is_available())
print("Versão do PyTorch:", torch.__version__)
print("Versão do NumPy:", np.__version__)
print("Versão do Transformers:", transformers.__version__)


CUDA disponível: True
Versão do PyTorch: 2.7.1+cu118
Versão do NumPy: 1.26.4
Versão do Transformers: 4.53.0


Carregar modelo e funçao de inferencia

In [24]:
import os
import subprocess
from f5_tts.infer.utils_infer import (
    infer_process,
    load_model,
    load_vocoder,
    remove_silence_for_generated_wav
)
from omegaconf import OmegaConf
from pathlib import Path
from datetime import datetime
import soundfile as sf
from hydra.utils import get_class
from pydub import AudioSegment

    # Load config & model
config_path = f"configs/F5TTS_v1_Base.yaml"
model_cfg = OmegaConf.load(config_path).model

model_cls = get_class(f"f5_tts.model.{model_cfg.backbone}")
model_arc = model_cfg.arch

ckpt_file="./model_last.safetensors"
vocoder_name="vocos"
model = load_model(
    model_cls, model_arc, ckpt_file, mel_spec_type=vocoder_name, vocab_file="./vocab.txt", device="cuda"
)
vocoder = load_vocoder(vocoder_name=vocoder_name)

def generate_tts(input_text_file, output_audio_file, ref_audio_map, nfe_step=38):
    temp_files = []
    os.makedirs("temp_audio", exist_ok=True)

    with open(input_text_file, "r", encoding="utf-8") as file:
        lines = file.readlines()

    for idx, line in enumerate(lines):
        if "]" not in line:
            print(f"Linha mal formatada (sem tag de referência): {line.strip()}")
            continue

        tag_part, gen_text = line.split("]", 1)
        tag = tag_part.strip().lstrip("[")  # Remove o "[" e espaços

        ref_audio = ref_audio_map.get(tag, {}).get("audio")
        ref_text = ref_audio_map.get(tag, {}).get("text")

        if not ref_audio:
            print(f"Referência de áudio '{tag}' não encontrada.")
            continue

        print(f"Gerando áudio para: {gen_text.strip()} com referência {ref_audio}")
        temp_file = f"temp_audio/temp_{idx}.wav"
        temp_files.append(temp_file)

        audio_segment, final_sample_rate, _ = infer_process(
            model_obj=model,
            vocoder=vocoder,
            ref_audio=ref_audio,
            ref_text=ref_text,
            gen_text=gen_text.strip(),
            speed=1,
            nfe_step=nfe_step,
            device="cuda"
        )

        sf.write(temp_file, audio_segment, final_sample_rate)

        # Adiciona silêncio no final
        sound = AudioSegment.from_wav(temp_file)
        silence = AudioSegment.silent(duration=300)
        combined = sound + silence
        combined.export(temp_file, format="wav")

    with open("file_list.txt", "w") as f:
        for temp_file in temp_files:
            f.write(f"file '{temp_file}'\n")

    subprocess.run(["ffmpeg", "-f", "concat", "-safe", "0", "-i", "file_list.txt", "-af", "highpass=f=200, lowpass=f=3000", "-acodec", "libmp3lame", output_audio_file], check=True)
    print(f"Áudio final salvo em {output_audio_file}")

    for temp_file in temp_files:
        os.remove(temp_file)
    os.remove("file_list.txt")
    os.rmdir("temp_audio")




vocab :  ./vocab.txt
token :  custom
model :  ./model_last.safetensors 

Download Vocos from huggingface charactr/vocos-mel-24khz


In [29]:
import os
import shutil

input_text_file = "input.txt"
output_audio_file = "output.mp3"
ref_audio_dir = "audio_ref"

# Mapeia as tags para arquivos de referência
ref_audio_map = {}
for file_name in os.listdir(ref_audio_dir):
    name, ext = os.path.splitext(file_name)
    path = os.path.join(ref_audio_dir, file_name)
    print(ref_audio_map)
    if ext == ".wav" or ext == "mp3":
        ref_audio_map.setdefault(name, {})["audio"] = path
    elif ext == ".txt":
        with open(path, "r", encoding="utf-8") as f:
            ref_audio_map.setdefault(name, {})["text"] = f.read().strip()

# Limpeza prévia
if os.path.exists("temp_audio"):
    shutil.rmtree("temp_audio")

if os.path.exists(output_audio_file):
    os.remove(output_audio_file)

os.makedirs("temp_audio", exist_ok=True)

# Executa TTS
generate_tts(input_text_file, output_audio_file, ref_audio_map,10)

from IPython.display import Audio
Audio(output_audio_file)


{}


TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [ ]:
from IPython.display import Audio

Audio("output.mp3")